In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [8]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Regular_Array'

# MakeModule Parameters
moduletype=simulationname
numpanels = 2
x = 2  # 2-up landscape
y = 1
xgap = 0.5
ygap = 0.5
zgap = 0.1
sensorsy = 9 


# TorqueTube Parameters
axisofrotationTorqueTube=True
torqueTube = True
tubetype = 'square'
diameter = 0.1
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=10
numcellsy=6

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 1  
nMods = 5
nRows = 3

azimuth_ang=210
tilt =30

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Noon, June 17th

# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': 3,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'Regular_Array')
REWRITING pre-existing module file. 
Module Regular_Array successfully created
Created Regular_Array.oct


In [3]:
name='Mycube'
text='! genbox black cuteBox 0.05 0.05 2.5 | xform -t -0.025 -0.025 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')

name='Mycube2'
text='! genbox black cuteBox 0.1 0.1 2.5 | xform -t -3.05 -0.05 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

('\nCustom Object Name', 'objects\\Mycube.rad')
('\nCustom Object Name', 'objects\\Mycube2.rad')
Created Regular_Array.oct


# At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:

rvu -vf views\front.vp -e .01 Regular_Array.oct

And then proceed happily with your analysis:

In [9]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
modWanted = 1
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(sceneDict['height'], sceneDict['azimuth'], sceneDict['tilt'], 
                                              sceneDict['pitch'], sceneDict['nMods'], sceneDict['nRows'], 
                                              moduleDict['sceney'], moduleDict['scenex'], moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=None)
analysis.analysis(octfile, "test3", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )

ModWanted, RowWanted  0 0  out of  5 3
Azimuth 210
Coordinate Center Point of Desired Panel before azm rotation -5.0 -3.0
Coordinate Center Point of Desired Panel after azm rotation 5.830127018922193 0.09807621135331557
Edge of Panel -0.5412658773652743 -0.9375 -0.6249999999999999
Offset Shift -0.037500000000000006 -0.06495190528383289 0.1299038105676658
Final Start Coordinate 5.251361141556918 -0.9043756939305173 0.504903810567666
Increase Coordinates 0.10825317547305487 0.1875 0.12499999999999999
Linescan in process: test3_Front
Linescan in process: test3_Back
Saved: results\irr_test3.csv
Annual bifacial ratio average:  0.473


EQUATIONS



In [ ]:
scenex = 2
sceney = 2.5
pitch = 4
nMods = 5
nRows = 3
tilt = 0
azimuth = 90
modWanted = 0#round (nMods/2.0)-1
rowWanted = 0#round (nRows/2.0)-1
offset = 0
height = 1
sensorsy = 9

import numpy as np
print nMods, nRows, modWanted, rowWanted

dtor = np.pi/180
x0 = (modWanted)*scenex - ((nMods-1)*scenex/2.0)
y0 = (rowWanted)*pitch - ((nRows-1)*pitch/2.0)

print "Coordinate Center Point of Desired Panel after azm rotation", x0,y0

x1 = x0 * np.cos (-azimuth*dtor) - y0 * np.sin(-azimuth*dtor)
y1 = x0 * np.sin (-azimuth*dtor) + y0 * np.cos(-azimuth*dtor)

print "Coordinate Center Point of Desired Panel after azm rotation", x1,y1

# Edge of Panel 
x2 = (sceney/2.0) * np.cos((tilt)*dtor) * np.sin((azimuth)*dtor)
y2 = (sceney/2.0) * np.cos((tilt)*dtor) * np.cos((azimuth)*dtor)
z2 = -(sceney/2.0) * np.sin(tilt*dtor)

print "Edge of Panel", x2, y2, z2

# Axis of rotation Offset (if offset is not 0)
x3 = offset * np.sin(tilt*dtor) * np.sin((azimuth)*dtor)
y3 = offset * np.sin(tilt*dtor) * np.cos((azimuth)*dtor)
z3 = offset * np.cos(tilt*dtor)

print "Offset Shift", x3, y3, z3

xstart = x1 + x2 + x3
ystart = y1 + y2 + y3
zstart = height + z2 + z3

print "Final Start Coordinate", xstart, ystart, zstart

xinc = -(sceney/(sensorsy + 1.0)) * np.cos((tilt)*dtor) * np.sin((azimuth)*dtor)
yinc = -(sceney/(sensorsy + 1.0)) * np.cos((tilt)*dtor) * np.cos((azimuth)*dtor) 
zinc = (sceney/(sensorsy + 1.0)) * np.sin(tilt*dtor) 

print "Increase Coordinates", xinc, yinc, zinc


In [ ]:
 moduleDict['moduleoffset']